### 그리드 서치(Grid Search)

* 모델 파라미터(일반적으로 말하는 파라미터를 의미): 모델이 학습 하면서 변화하게 되는 값. 딥러닝 모델의 경우 가중치가 파라미터

* 하이퍼 파라미터(매개변수): 모델의 학습 전에 설정해 주는 값. 아무런 설정도 하지 않으면 기본값(`default`)로 학습하게 됨

**GridSearchCV** 는 k-fold cross validation 을 해준다

In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x") # svg, retina, png2x ...
mpl.style.use("seaborn-v0_8")
mpl.rcParams.update({"figure.constrained_layout.use": True})
sns.set_context("paper") 
sns.set_palette("Set2") 
sns.set_style("whitegrid") 

# 시스템 폰트패밀리에 따라 변경
plt.rc("font", family = "NanumSquareRound")
plt.rcParams["axes.unicode_minus"] = False

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_csv("../../data/heart.csv")
X = data.drop(columns="age")
y = data["age"]

x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3,
                                                    random_state=42)

In [23]:
# max_depth = 3
model = DecisionTreeClassifier(max_depth = 3, random_state=209) 
model.fit(x_train.values, y_train.values)
prediction = model.predict(x_test.values)
print('The accuracy of the DT is', 
      accuracy_score(prediction, y_test))

The accuracy of the DT is 0.054945054945054944


* 따라서 그리드서치를 할 때에는 로 하는 것이 좋다.

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
params = {"max_depth" : [3, 4, 5, 10]} # params 갯수 * cv 3 = 12 번의 학습

model_base = DecisionTreeClassifier(random_state=209) 
model_grid = GridSearchCV(model_base, 
                          params, 
                          cv=3, 
                          return_train_score=True,
                          n_jobs = -1 # `n_jobs = -1`(모든 코어 사용)
                         )
model_grid.fit(x_train, y_train)

/home/tommy/miniconda3/envs/p311/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=209),
             n_jobs=-1, param_grid={'max_depth': [3, 4, 5, 10]},
             return_train_score=True)

In [26]:
# 가장 결과가 좋은 매개변수(하이퍼 파라미터)
model_grid.best_params_

{'max_depth': 3}

In [27]:
# 훈련데이터 중 가장 좋은 정확도
model_grid.best_score_

np.float64(0.0471495640509725)

In [28]:
# 가장 결과가 좋은 매개변수로 테스트 데이터를 예측해 보자
model_best = model_grid.best_estimator_
model_best.predict(x_test)

array([57, 57, 43, 60, 54, 43, 54, 60, 58, 57, 57, 60, 43, 58, 54, 54, 43,
       57, 58, 58, 57, 57, 57, 57, 57, 54, 54, 54, 57, 54, 57, 60, 58, 58,
       58, 57, 60, 54, 43, 54, 54, 54, 54, 60, 54, 54, 57, 54, 60, 57, 54,
       54, 54, 54, 54, 60, 60, 54, 60, 60, 58, 54, 57, 60, 57, 57, 54, 54,
       57, 54, 54, 54, 57, 54, 60, 54, 54, 60, 57, 57, 54, 58, 57, 57, 54,
       60, 58, 60, 57, 57, 57])

In [29]:
## model_best의 테스트 데이터에 대한 정확도를 계산하시오.
pred_best = model_best.predict(x_test)
accuracy_score(pred_best, y_test)

0.054945054945054944

In [30]:
def grid_search(params, model_base=DecisionTreeClassifier(random_state=209)):
    model_grid = GridSearchCV(model_base, 
                              params, 
                              cv=3, 
                              return_train_score=True,
                              #n_jobs = -1
                             )
    model_grid.fit(x_train, y_train)
    
    print('최상의 매개변수: ', model_grid.best_params_)
    print('훈련 데이터의 최고 정확도: ', model_grid.best_score_)
    
    model_best = model_grid.best_estimator_
    pred_best = model_best.predict(x_test)
    print('테스트 데이터의 최고 정확도: ', accuracy_score(pred_best, y_test))

In [31]:
grid_search(params = {"min_samples_leaf": [1,2,4,8],
                     "max_depth" : [3, 4, 5, 10],
                      "criterion": ["gini", "entropy"]})

/home/tommy/miniconda3/envs/p311/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


최상의 매개변수:  {'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 8}
훈련 데이터의 최고 정확도:  0.05674044265593561
테스트 데이터의 최고 정확도:  0.054945054945054944


### RandomizedSearchCV
* 작동방식 : GridSearchCV와 동일, 그러나 하이퍼 파라미터를 랜덤하게 조합

In [32]:
from sklearn.model_selection import RandomizedSearchCV

In [33]:
def random_grid_search(params, iters, model_base=DecisionTreeClassifier(random_state=209)):
    model_grid = RandomizedSearchCV(model_base, 
                                    params,
                                    n_iter = iters,
                                    cv=3, 
                                    return_train_score=True,
                                    #n_jobs = -1,
                                    random_state = 97
                                    )
    model_grid.fit(x_train, y_train)
    
    print('최상의 매개변수: ', model_grid.best_params_)
    print('훈련 데이터의 최고 정확도: ', model_grid.best_score_)
    
    model_best = model_grid.best_estimator_
    pred_best = model_best.predict(x_test)
    print('테스트 데이터의 최고 정확도: ', accuracy_score(pred_best, y_test))

In [34]:
params = {"criterion": ["entropy", "gini"],
          "min_samples_leaf": np.random.randint(1, 10, 10),
          "max_depth" : np.random.randint(3, 8, 20),
          "min_impurity_decrease": np.random.uniform(0.001, 0.01, 30)}

In [35]:
random_grid_search(params, 50, model_base)

/home/tommy/miniconda3/envs/p311/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


최상의 매개변수:  {'min_samples_leaf': np.int64(8), 'min_impurity_decrease': np.float64(0.00972951671429884), 'max_depth': np.int64(7), 'criterion': 'gini'}
훈련 데이터의 최고 정확도:  0.07082494969818913
테스트 데이터의 최고 정확도:  0.054945054945054944
